### 임상시험 데이터셋 모델
임상시험 데이터셋 표준인 CDASH, SDTM에 대해 알게 된 내용을 정리함.

#### CDASH, SDTM
CDASH는 CRF를 통해 획득된 데이터를 0차적으로 정의하고 SDTM으로 기계 변환 가능하게 하는 규칙임. 임상시험용 앱 프론트앤드서 필드에 입력된 데이터가 백앤드에 어떻게 저장되는지를 규정하고, 기정의 디자인 요소를 제공함.

SDTM은 FDA 제출용 데이터셋으로 표준화된 규칙에 의거 변수의 이름과 내용을 정하고 카테고레이즈할 수 있는 프로토콜임.

#### 특징
- CRF에서 생일 필드를 년, 월, 일의 문자열로 텍스트 박스에 입력하면, 각각 CDASH (BRTHYR, BRTHMO, BRTHDY)의 변수(컬럼)으로 냄
    - SDTM ( BRTHDTC = (BRTHYR + BRTHMO + BRTHDY) )으로 파싱함
- CDASH에는 16개 도메인이 있음
    - ("AE", "CO", "CM", "DM", "DS", "DA", "EG", "EX", "IE", "LB", "MH", "PE", "DV", "SC", "SU", "VS")
    - 도메인마다 CRF 페이지가 구성되는 것은 아님
- CDASH로 모든 CRF 항목이 커버되지 않으며 CDASH를 벗어날수록 SDTM 변환이 어려워질 것임
    - 이러면 SUPP 데이터셋 사용 필요
- 컬럼 값은 여러 CRF 필드의 파싱 결과이며 필드와 반드시 1:1 대응되지는 않음
- SDTM 데이터셋은 롱 포맷(vertical)임
    - 각 데이터셋에서 인덱스(key)로 사용되는 컬럼이 규정됨
        - {"key": {"CM": "STUDYID", "USUBJID", "CMTRT", "CMSTDTC"}}
    - 각 데이터셋의 키 컬럼을 인덱스로 했을 때 개수가 레코드 개수임

#### VS를 SDTM에 맞추기
[dmisimportant 연습용 데이터셋](https://cafe.naver.com/dmisimportant/104)의 VS를 SDTM 비슷하게 만들고자 함.

- 필요 모듈 로드, 스펙시트 경로 설정
- 스펙시트를 매퍼(딕셔너리)로서 읽음
    - 예전 노트북에서의 스펙시트 클래스를 딕셔너리 매퍼를 바로 내도록 바꿈
- sas7bdat 로드

In [1]:
import os
from dm import *
specpath="C:/code/CUBEDEMO2017/spec.xlsx"
datapath="C:/code/CUBEDEMO2017/SASSET/"
spec=Spec(pd.read_excel(specpath))
sdtm=pd.read_excel(specpath,sheet_name=1).set_index(keys="Domain Prefix")

sasobj=[obj for obj in os.scandir(datapath) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
data={os.path.splitext(obj.name)[0].upper():read_sas_(obj.path) for obj in sasobj}
print(ornament,"domain:\n",data.keys(),len(data),"domains")

spec.map[("EN","VERSION")]

c:\code\pub\ct\dm.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spec["CODE"]=[dict(q.split(":") for q in w.split("|")) if isinstance(w,str) else w for w in spec.CODE]


---------- domain:
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 27 domains


{'PGNM': 'EN',
 'PGNO': 1,
 'PAGE_LABEL': 'Enrollment',
 'CRF_LABEL': 'Enrollment',
 'VISIT': '0',
 'ITEM_SEQ': 2,
 'ITEM_LABEL': 'Version',
 'CODE': nan,
 'LAYOUT': 'SYSDEFINED',
 'KEY': 1.0,
 'TYPE_LENGTH': 'C10',
 'VIEW_TYPE': 'nvarchar2(10)'}

In [2]:
sdtm.loc["VS"].query("Core!='Perm'")

,Seq. For Order,Observation Class,Variable Name \n(minus domain prefix),Variable Name,Variable Label,Type,"Controlled Terms, Codelist or Format",Role,CDISC Notes (for domains)\nDescription (for General Classes),Core,References
Domain Prefix,,,,,,,,,,,
VS,1,Findings,STUDYID,STUDYID,Study Identifier,Char,NaN,Identifier,Unique identifier for a study.,Req,SDTM 2.2.4
VS,2,Findings,DOMAIN,DOMAIN,Domain Abbreviation,Char,VS,Identifier,Two-character abbreviation for the domain.,Req,"SDTM 2.2.4, SDTMIG 4.1.2.2, SDTMIG Appendix C2"
VS,3,Findings,USUBJID,USUBJID,Unique Subject Identifier,Char,NaN,Identifier,Identifier used to uniquely identify a subject...,Req,"SDTMIG 2.2.4, SDTMIG 4.1.2.3"
VS,4,Findings,SEQ,VSSEQ,Sequence Number,Num,NaN,Identifier,Sequence Number given to ensure uniqueness of ...,Req,SDTM 2.2.4
VS,7,Findings,TESTCD,VSTESTCD,Vital Signs Test Short Name,Char,(VSTESTCD),Topic,"Short name of the measurement, test, or examin...",Req,"SDTM 2.2.3, SDTMIG 4.1.1.8, SDTMIG 4.1.2.1, SD..."
VS,8,Findings,TEST,VSTEST,Vital Signs Test Name,Char,(VSTEST),Synonym Qualifier,Verbatim name of the test or examination used ...,Req,"SDTM 2.2.3, SDTMIG 4.1.2.1, SDTMIG 4.1.2.4, SD..."
VS,12,Findings,ORRES,VSORRES,Result or Finding in Original Units,Char,NaN,Result Qualifier,Result of the vital signs measurement as origi...,Exp,"SDTM 2.2.3, SDTMIG 4.1.5.1"
VS,13,Findings,ORRESU,VSORRESU,Original Units,Char,(VSRESU),Variable Qualifier,Original units in which the data were collecte...,Exp,"SDTM 2.2.3, SDTMIG 4.1.3.2, SDTMIG 4.1.5.1, SD..."
VS,14,Findings,STRESC,VSSTRESC,Character Result/Finding in Std Format,Char,NaN,Result Qualifier,"Contains the result value for all findings, co...",Exp,"SDTM 2.2.3, SDTMIG 4.1.5.1"


- cubeCDMS VS 데이터셋 원본을 SDTMIG의 VS 인덱스(keys)에 맞추어 스태킹한다.

In [3]:
vs=data["VS"]
vs0=vs.set_index(["SUBJID","VISIT","VSYN","VSDTC"]).stack(dropna=False).reset_index()
vs0

,SUBJID,VISIT,VSYN,VSDTC,level_4,0
0,S-1Z-005,1.0,1.0,2016-02-10,HEIGHT,200.0
1,S-1Z-005,1.0,1.0,2016-02-10,WEIGHT,115.0
2,S-1Z-005,1.0,1.0,2016-02-10,SYSBP,170.0
3,S-1Z-005,1.0,1.0,2016-02-10,DIABP,120.0
4,S-1Z-005,1.0,1.0,2016-02-10,PULSE,120.0
...,...,...,...,...,...,...
2403,S-US-007,1.0,1.0,2017-03-01,SYSBP,20.0
2404,S-US-007,1.0,1.0,2017-03-01,DIABP,160.0
2405,S-US-007,1.0,1.0,2017-03-01,PULSE,30.0
2406,S-US-007,1.0,1.0,2017-03-01,RESP,25.0


- SDTM의 VS 필수 컬럼(Core가 Permissible이 아닌 것)을 낸다.

In [4]:
cols=list(sdtm.loc["VS"].query("Core!='Perm'")["Variable Name"])
cols

['STUDYID',
 'DOMAIN',
 'USUBJID',
 'VSSEQ',
 'VSTESTCD',
 'VSTEST',
 'VSORRES',
 'VSORRESU',
 'VSSTRESC',
 'VSSTRESN',
 'VSSTRESU',
 'VSBLFL',
 'VISITNUM',
 'VSDTC']

- 각 컬럼의 내용을 정의한다.
- 상수가 아닌 리스트나 시리즈의 인덱스는 vs0의 인덱스를 따른다.
- 단위나 Score String (NORMAL, ABNORMAL 등)은 프로토콜을 모르므로 임의의 문자열로 설정한다.
    - 향후 프로토콜에 따라 매퍼를 설정하고 맞는 값으로 바꿀 수 있다.

In [5]:
siteid="0"
studyid="0"
domain="VS"
usubjid=studyid+"-"+vs0.SUBJID+"-"+siteid
vsseq=vs0.groupby(["SUBJID","VISIT","VSYN","VSDTC"]).cumcount()+1
vtestcd=vs0.level_4.copy()
vstest=vs0.level_4.apply(lambda q:spec.map[("VS",q)]["ITEM_LABEL"].upper())
vsorres=vs0[0].copy()
vsblfl=(vs0.VISIT==1).replace({True:"Y",False:np.nan})

vsorresu="_unit"
vsstresc="_scale"
vsstresn=vsorres
vsstresu=vsorresu

visitnum=vs0.VISIT
vsdtc=vs0.VSDTC

- 생성된 컬럼(상수, 리스트, series)을 결합한다.

In [6]:
vs_sdtm=pd.DataFrame(dict(zip(cols,[studyid,domain,usubjid,vsseq,vtestcd,vstest,vsorres,vsorresu,vsstresc,vsstresn,vsstresu,vsblfl,visitnum,vsdtc])))
vs_sdtm

,STUDYID,DOMAIN,USUBJID,VSSEQ,VSTESTCD,VSTEST,VSORRES,VSORRESU,VSSTRESC,VSSTRESN,VSSTRESU,VSBLFL,VISITNUM,VSDTC
0,0,VS,0-S-1Z-005-0,1.0,HEIGHT,HEIGHT,200.0,_unit,_scale,200.0,_unit,Y,1.0,2016-02-10
1,0,VS,0-S-1Z-005-0,2.0,WEIGHT,WEIGHT,115.0,_unit,_scale,115.0,_unit,Y,1.0,2016-02-10
2,0,VS,0-S-1Z-005-0,3.0,SYSBP,SYSTOLIC BLOOD PRESSURE,170.0,_unit,_scale,170.0,_unit,Y,1.0,2016-02-10
3,0,VS,0-S-1Z-005-0,4.0,DIABP,DIASTOLIC BLOOD PRESSURE,120.0,_unit,_scale,120.0,_unit,Y,1.0,2016-02-10
4,0,VS,0-S-1Z-005-0,5.0,PULSE,PULSE RATE,120.0,_unit,_scale,120.0,_unit,Y,1.0,2016-02-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2403,0,VS,0-S-US-007-0,3.0,SYSBP,SYSTOLIC BLOOD PRESSURE,20.0,_unit,_scale,20.0,_unit,Y,1.0,2017-03-01
2404,0,VS,0-S-US-007-0,4.0,DIABP,DIASTOLIC BLOOD PRESSURE,160.0,_unit,_scale,160.0,_unit,Y,1.0,2017-03-01
2405,0,VS,0-S-US-007-0,5.0,PULSE,PULSE RATE,30.0,_unit,_scale,30.0,_unit,Y,1.0,2017-03-01
2406,0,VS,0-S-US-007-0,6.0,RESP,RESPIRATORY RATE,25.0,_unit,_scale,25.0,_unit,Y,1.0,2017-03-01


- 무결성 확인 1
    - VSSEQ: 1회 방문시 측정될 VS 항목의 순번: 빈 값을 무시하지 않도록 했으므로 각 개수가 같아야 한다.
    - VSBLFL: VS Check의 Baseline 여부: "Y"면 VISITNUM이 1이어야 한다.

In [7]:
print(vs_sdtm.VSSEQ.value_counts())
print(vs_sdtm[vs_sdtm.VSBLFL=="Y"].VISITNUM.value_counts())

1.0    236
2.0    236
3.0    236
4.0    236
5.0    236
6.0    236
7.0    236
Name: VSSEQ, dtype: int64
1.0    700
Name: VISITNUM, dtype: int64


- 무결성 확인 2
    - vs0의 VSYN은 VS Check 여부이다. 
    - 원본 데이터셋에는 VSYN이 1이고 VSDTC가 빈 값이며 SYSBP만 입력된 경우가 있다. 이 레코드들이 있어야 한다.

In [8]:
vsyn=vs0.VSYN==1
vs_sdtm[vsyn][pd.isna(vs_sdtm[vsyn].VSDTC)].query("(VSTESTCD=='SYSBP') & (VSORRES>0)").sample(5)

,STUDYID,DOMAIN,USUBJID,VSSEQ,VSTESTCD,VSTEST,VSORRES,VSORRESU,VSSTRESC,VSSTRESN,VSSTRESU,VSBLFL,VISITNUM,VSDTC
1507,0,VS,0-S-3Z-025-0,NaN,SYSBP,SYSTOLIC BLOOD PRESSURE,170.0,_unit,_scale,170.0,_unit,NaN,5.0,NaN
1941,0,VS,0-S-5Z-003-0,NaN,SYSBP,SYSTOLIC BLOOD PRESSURE,150.0,_unit,_scale,150.0,_unit,NaN,2.0,NaN
1759,0,VS,0-S-3Z-049-0,NaN,SYSBP,SYSTOLIC BLOOD PRESSURE,150.0,_unit,_scale,150.0,_unit,NaN,5.0,NaN
2312,0,VS,0-S-MJ-013-0,NaN,SYSBP,SYSTOLIC BLOOD PRESSURE,150.0,_unit,_scale,150.0,_unit,NaN,5.0,NaN
1787,0,VS,0-S-3Z-052-0,NaN,SYSBP,SYSTOLIC BLOOD PRESSURE,150.0,_unit,_scale,150.0,_unit,NaN,2.0,NaN


#### AE를 SDTM에 맞추기
- AE는 원래부터 롱 포맷이기에 간단히 후처리할 수 있다.

In [29]:
ae=data["AE"]
ae.set_index(keys=["SUBJID","VISIT","SEQ"])[:5]

AETERM     AESTDTC  AETEAE  AEOUT     AEENDTC  \
SUBJID   VISIT  SEQ                                                        
S-1Z-001 3003.0 1.0          TEST  2017-02-01     1.0    2.0  2017-02-16   
                2.0          Cold  2017-02-02     1.0    6.0  2017-02-14   
S-1Z-002 3003.0 1.0          asdf  2017-02-08     NaN    5.0         NaN   
S-1Z-005 3003.0 1.0  Liver damage  2016-06-16     1.0    1.0  2016-06-17   
                2.0        Asthma  2016-06-18     1.0    4.0  2016-06-22   

                     AESER  AESEV  AEREL  AEACN  AEACNOTH  ...  HLGTCD SOCCD  \
SUBJID   VISIT  SEQ                                        ...                 
S-1Z-001 3003.0 1.0    4.0    3.0    2.0    3.0       3.0  ...     NaN   NaN   
                2.0    2.0    1.0    4.0    4.0       1.0  ...     NaN   NaN   
S-1Z-002 3003.0 1.0    NaN    NaN    NaN    NaN       NaN  ...     NaN   NaN   
S-1Z-005 3003.0 1.0    4.0    2.0    2.0    2.0       2.0  ...     NaN   NaN   
                2.0    3.0    2.0    3.0    3.0       2.0  ...     NaN   NaN   

                     PSFG CODINGEXP CODINGCMT       INV_LLT           INV_PT  \
SUBJID   VISIT  SEQ                                                            
S-1Z-001 3003.0 1.0   NaN       NaN       NaN           NaN              NaN   
                2.0   NaN         -       NaN          Cold  Nasopharyngitis   
S-1Z-002 3003.0 1.0   NaN       NaN       NaN           NaN              NaN   
S-1Z-005 3003.0 1.0   NaN       NaN       NaN  Liver damage     Liver injury   
                2.0   NaN       NaN       NaN        Asthma           Asthma   

                                                     INV_HLT   INV_LLTCD  \
SUBJID   VISIT  SEQ                                                        
S-1Z-001 3003.0 1.0                                      NaN         NaN   
                2.0       Upper respiratory tract infections  10009851.0   
S-1Z-002 3003.0 1.0                                      NaN         NaN   
S-1Z-005 3003.0 1.0  Hepatocellular damage and hepatitis NEC  10024668.0   
                2.0             Bronchospasm and obstruction  10003553.0   

                     INV_HLGTCD  
SUBJID   VISIT  SEQ              
S-1Z-001 3003.0 1.0         NaN  
                2.0  10021879.0  
S-1Z-002 3003.0 1.0         NaN  
S-1Z-005 3003.0 1.0  10019654.0  
                2.0  10006436.0  

[5 rows x 34 columns]

- 예전 노트북에서는 아무 데이터셋에서 SUBJID 등 인덱스로 쓸만한 컬럼을 냈지만, 이 연습용 데이터셋은 실험대상자 / 입력 항목의 부분적인 내용만을 담고 있음을 알았으므로 각 도메인 데이터셋의 SUBJID를 사용한다.

In [27]:
domain="AE"
usubjid=studyid+"-"+ae.SUBJID+"-"+siteid
aeseq=ae.SEQ
aeterm=ae.AETERM
aellt=ae.INV_LLT
aelltcd=ae.INV_LLTCD
aedecod=ae.INV_PT
aeptcd=ae.INV_PTCD
aehlt=ae.INV_HLT
aehltcd=ae.INV_HLTCD
aehlgt=ae.INV_HLGT
aehlgtcd=ae.INV_HLGTCD
aebodsys=ae.INV_SOC
aebdsycd=ae.INV_SOCCD
aesoc=ae.INV_SOC
aesoccd=ae.INV_SOCCD
aeser=ae.AESER.apply(lambda q:spec.map[("AE","AESER")]["CODE"][str(int(q))].upper() if pd.notna(q) else q)
aeacn=ae.AEACN.apply(lambda q:spec.map[("AE","AEACN")]["CODE"][str(int(q))].upper() if pd.notna(q) else q)
aerel=ae.AEREL.apply(lambda q:spec.map[("AE","AEREL")]["CODE"][str(int(q))].upper() if pd.notna(q) else q)
aestdtc=ae.AESTDTC
aeendtc=ae.AEENDTC

In [28]:
cols=list(sdtm.loc["AE"].query("Core!='Perm'")["Variable Name"])
ae_sdtm=pd.DataFrame(dict(zip(cols,[studyid,domain,usubjid,aeseq,aeterm,aellt,aelltcd,aedecod,aeptcd,aehlt,aehltcd,aehlgt,aehlgtcd,aebodsys,aebdsycd,aesoc,aesoccd,aeser,aeacn,aerel,aestdtc,aeendtc])))
ae_sdtm

,STUDYID,DOMAIN,USUBJID,AESEQ,AETERM,AELLT,AELLTCD,AEDECOD,AEPTCD,AEHLT,...,AEHLGTCD,AEBODSYS,AEBDSYCD,AESOC,AESOCCD,AESER,AEACN,AEREL,AESTDTC,AEENDTC
0,0,AE,0-S-1Z-001-0,1.0,TEST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,LIFE THREATENING,DOSE REDUCED,UNLIKELY RELATED,2017-02-01,2017-02-16
1,0,AE,0-S-1Z-001-0,2.0,Cold,Cold,10009851.0,Nasopharyngitis,10028810.0,Upper respiratory tract infections,...,10021879.0,Infections and infestations,10021881.0,Infections and infestations,10021881.0,DEATH,DRUG INTERRUPTED,RELATED,2017-02-02,2017-02-14
2,0,AE,0-S-1Z-002-0,1.0,asdf,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-02-08,NaN
3,0,AE,0-S-1Z-005-0,1.0,Liver damage,Liver damage,10024668.0,Liver injury,10067125.0,Hepatocellular damage and hepatitis NEC,...,10019654.0,Hepatobiliary disorders,10019805.0,Hepatobiliary disorders,10019805.0,LIFE THREATENING,DOSE NOT CHANGED,UNLIKELY RELATED,2016-06-16,2016-06-17
4,0,AE,0-S-1Z-005-0,2.0,Asthma,Asthma,10003553.0,Asthma,10003553.0,Bronchospasm and obstruction,...,10006436.0,"Respiratory, thoracic and mediastinal disorders",10038738.0,"Respiratory, thoracic and mediastinal disorders",10038738.0,HOSPITALIZATION,DOSE REDUCED,POSSIBLY RELATED,2016-06-18,2016-06-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,0,AE,0-S-MJ-011-0,1.0,test,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NO,DOSE NOT CHANGED,NOT RELATED,2017-05-01,NaN
187,0,AE,0-S-MJ-012-0,1.0,Acute nephritis,Acute nephritis,10000940.0,Nephritis,10029117.0,Nephritis NEC,...,10029149.0,Renal and urinary disorders,10038359.0,Renal and urinary disorders,10038359.0,HOSPITALIZATION,DRUG WITHDRAWN,UNLIKELY RELATED,2016-07-07,NaN
188,0,AE,0-S-MJ-013-0,2.0,Death,Death,10011906.0,Death,10011906.0,Death and sudden death,...,10053172.0,General disorders and administration site cond...,10018065.0,General disorders and administration site cond...,10018065.0,NO,UNKNOWN,UNLIKELY RELATED,2017-06-04,2017-06-05
189,0,AE,0-S-US-002-0,1.0,Diarrhea,Diarrhea,10012727.0,Diarrhoea,10012735.0,Diarrhoea (excl infective),...,10017977.0,Gastrointestinal disorders,10017947.0,Gastrointestinal disorders,10017947.0,OTHER MEDICALLY IMPORTANT EVENT,DRUG WITHDRAWN,POSSIBLY RELATED,2017-03-21,NaN


- 타입 업캐스팅 주의: 코드 매퍼가 적용될 컬럼(e.g. AESER, VSTEST)은 dtype을 오브젝트로 내고, 최종 데이터는 xport 변환 직전 타입 변환 필요

#### 후기
- 전반적인 파악을 위해 비절차적으로 컬럼 값을 내고 결합했다.
    - CRF 디자인이 표준화됐을수록 매퍼를 통해 비절차적인 진행이 가능하며, 서로 다른 프로토콜의 임상시험에 대해 공수 투입을 최소화하며 SDTM 후처리가 가능하다.
- DB 스펙시트가 잘 되어 있어 기계적으로 진행 가능: XML보다 cubeCDMS 스펙시트가 좋음
- 궁극적으로 SDTM 기반 후처리는 CDISC SHARE API에서 스펙을 받아 써야함